In [ ]:
# import packages

import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn import svm
from sklearn import neighbors
import torch
from torch import nn, optim
from skorch import NeuralNetClassifier
from skorch.callbacks import EpochScoring 
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import pandas as pd
import multiprocessing
import seaborn as sns

In [ ]:
# set random seed
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)
device = 'cpu'

In [ ]:
# define neural network class, code adapted from https://github.com/jlm429/pyperch/blob/master/pyperch/neural/backprop_nn.py

class BackpropModule(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_units=10, hidden_layers=1,
                 dropout_percent=0, activation=nn.ReLU(), output_activation=nn.Softmax(dim=-1)):
        """

        Initialize the neural network.

        PARAMETERS:

        input_dim {int}:
            Number of features/dimension of the input.  Must be greater than 0.

        output_dim {int}:
            Number of classes/output dimension of the model. Must be greater than 0.

        hidden_units {int}:
            Number of hidden units.

        hidden_layers {int}:
            Number of hidden layers.

        dropout_percent {float}:
            Probability of an element to be zeroed.

        activation {torch.nn.modules.activation}:
            Activation function.

        output_activation {torch.nn.modules.activation}:
            Output activation.

        """
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_units = hidden_units
        self.hidden_layers = hidden_layers
        self.dropout = nn.Dropout(dropout_percent)
        self.activation = activation
        self.output_activation = output_activation
        self.layers = nn.ModuleList()
        self.device = "cpu"

        # input layer
        self.layers.append(nn.Linear(self.input_dim, self.hidden_units, device=self.device))
        # hidden layers
        for layer in range(self.hidden_layers):
            self.layers.append(nn.Linear(self.hidden_units, self.hidden_units, device=self.device))
        # output layer
        self.layers.append(nn.Linear(self.hidden_units, self.output_dim, device=self.device))

    def forward(self, X, **kwargs):
        """
        Recipe for the forward pass.

        PARAMETERS:

        X {torch.tensor}:
            NN input data. Shape (batch_size, input_dim).

        RETURNS:

        X {torch.tensor}:
            NN output data. Shape (batch_size, output_dim).
        """
        X = self.activation(self.layers[0](X))
        X = self.dropout(X)
        for i in range(self.hidden_layers):
            X = self.activation(self.layers[i+1](X))
            X = self.dropout(X)
        X = self.output_activation(self.layers[self.hidden_layers+1](X))
        return X

In [ ]:
# Load bean dataset, binary classification with larger dataset
 
# fetch dataset 
bean = pd.read_excel('.//datasets//bean//Dry_Bean_Dataset.xlsx')

X_bean = bean.iloc[:, :-1]
y_bean = bean.iloc[:, -1]

encoder = preprocessing.LabelEncoder().fit(y_bean)
y_bean = encoder.transform(y_bean)

X_bean_train, X_bean_test, y_bean_train, y_bean_test = model_selection.train_test_split(X_bean, y_bean, test_size=0.2, random_state=seed, stratify=y_bean)

scaler_bean = preprocessing.StandardScaler().fit(X_bean_train)
X_bean_train = scaler_bean.transform(X_bean_train)
X_bean_test = scaler_bean.transform(X_bean_test)

X_train = X_bean_train.astype(np.float32)
X_test = X_bean_test.astype(np.float32)
y_train = y_bean_train.astype(np.int64)
y_test = y_bean_test.astype(np.int64)

bean_y = bean.copy()
bean_y.iloc[:, :-1] = scaler_bean.transform(bean_y.iloc[:, :-1])
bean_y['Class'] = y_bean

In [ ]:
col_names = X_bean.columns
counts = bean['Class'].value_counts().values

plt.figure(figsize=(4.8,4.8))
plt.bar(np.unique(bean['Class']), counts, width=0.5)
plt.xticks(rotation=45)
plt.ylabel('Count')

fig, axes = plt.subplots(4, 4, figsize=(16, 14))
axes = axes.flatten()
for i, col in enumerate(col_names):
    sns.kdeplot(data=bean_y, x=col, hue='Class', ax=axes[i], common_norm=False)
    axes[i].set_title(f'{col} distribution');
    axes[i].set_xlabel(None)
plt.tight_layout()
plt.rcParams.update({'font.size': 12})
# plt.savefig('bean_feature.png', dpi=500)

In [ ]:
# initialize neural net for wine dataset and perform gridsearch

num_features = X_train.shape[1]
num_classes = len(np.unique(y_train))

NN = NeuralNetClassifier(
    module=BackpropModule,
    module__input_dim=num_features,
    module__output_dim=num_classes,
    module__hidden_units=128,
    module__hidden_layers=2,
    module__dropout_percent=0,
    max_epochs=500,
    verbose=0,
    callbacks=[EpochScoring(scoring='accuracy', name='train_acc', on_train=True),],
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.SGD,
    optimizer__weight_decay=0,
    optimizer__momentum=0,
    lr=0.05,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    device=device
)

NN.set_params(train_split=False, verbose=0)

default_params = {
    'module__input_dim': [num_features],
    'module__output_dim': [num_classes],
    'max_epochs': [500]
}

params = {
    'lr': [0.001, 0.01, 0.1],
    'module__hidden_units': [16, 32, 64, 128, 256],
    'module__hidden_layers': [1, 2, 3],
    'module__dropout_percent': [0, 0.1, 0.2, 0.3],
    'optimizer__weight_decay': [0, 1e-4, 1e-3, 1e-2],
    'optimizer__momentum': [0, 0.9], 
    **default_params,
}

folds = StratifiedKFold(n_splits=5)
gs_NN = GridSearchCV(NN, params, refit=False, cv=folds, scoring='accuracy', verbose=3, n_jobs=12)

gs_NN.fit(X_train, y_train)

df = pd.DataFrame(gs_NN.cv_results_)
df.to_excel('NN_bean_temp.xlsx')

In [ ]:
# Extract best params for NN 

NN_best_param = gs_NN.best_params_
print(NN_best_param)

In [ ]:
NN_best = NeuralNetClassifier(
    module=BackpropModule,
    module__input_dim=num_features,
    module__output_dim=num_classes,
    module__hidden_units=256,
    module__hidden_layers=2,
    module__dropout_percent=0.1,
    max_epochs=200,
    verbose=0,
    callbacks=[EpochScoring(scoring='accuracy', name='train_acc', on_train=True),],
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.SGD,
    optimizer__weight_decay=0,
    optimizer__momentum=0.9,
    lr=0.01,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

# Vary architechture to understand its effects:
default_params = {
    'module__input_dim': [num_features],
    'module__output_dim': [num_classes],
    'max_epochs': [500],
    'module__dropout_percent': [0.1],
    'optimizer__weight_decay': [0],
    'optimizer__momentum': [0.9],
    'lr': [0.01]
}

params = {
    'module__hidden_units': [8, 16, 32, 64, 128, 256],
    'module__hidden_layers': [0, 1, 2, 3],
    **default_params,
}

NN_best_arch = GridSearchCV(NN_best, params, refit=False, cv=folds, scoring='accuracy', verbose=3, n_jobs=12)

NN_best_arch.fit(X_train, y_train)

df = pd.DataFrame(NN_best_arch.cv_results_)
df.to_excel('NN_bean_arch.xlsx')

In [ ]:
# Vary dropout and regularization to understand its effects:
default_params = {
    'module__input_dim': [num_features],
    'module__output_dim': [num_classes],
    'max_epochs': [500],
    'optimizer__momentum': [0.9],
    'lr': [0.01],
    'module__hidden_units': [64],
    'module__hidden_layers': [1]
}

params = {
    'module__dropout_percent': [0, 0.1, 0.2, 0.3],
    'optimizer__weight_decay': [0, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3],
    **default_params,
}

NN_best_reg = GridSearchCV(NN_best, params, refit=False, cv=folds, scoring='accuracy', verbose=3, n_jobs=12)

NN_best_reg.fit(X_train, y_train)

df = pd.DataFrame(NN_best_reg.cv_results_)
df.to_excel('NN_bean_reg.xlsx')

In [ ]:
# Vary lr to understand its effects:

NN_lr_results = {}

for lr_range in [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]:
    NN_lr = NeuralNetClassifier(
        module=BackpropModule,
        module__input_dim=num_features,
        module__output_dim=num_classes,
        module__hidden_units=64,
        module__hidden_layers=1,
        module__dropout_percent=0.1,
        max_epochs=500,
        verbose=0,
        callbacks=[EpochScoring(scoring='accuracy', name='train_acc', on_train=True),],
        criterion=nn.CrossEntropyLoss,
        optimizer=optim.SGD,
        optimizer__weight_decay=0,
        optimizer__momentum=0.9,
        lr=lr_range,
        # Shuffle training data on each epoch
        iterator_train__shuffle=True,
    )
    
    NN_lr.fit(X_train, y_train)

    NN_lr_results[str(lr_range)+'_train'] = NN_lr.history[:, 'train_acc']
    NN_lr_results[str(lr_range)+'_valid'] = NN_lr.history[:, 'valid_acc']

df = pd.DataFrame(NN_lr_results)
df.to_excel('NN_bean_lr2.xlsx')

In [ ]:
# Hyperparam plots

NN_arch_data = pd.read_excel('NN_bean_arch.xlsx')
NN_reg_data = pd.read_excel('NN_bean_reg.xlsx')

units =  [8, 16, 32, 64, 128, 256]
layers = [1, 2, 3]

decay =  [1e-6, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3]
dps = [0, 0.1, 0.2, 0.3]

fig, ax = plt.subplots(2, 2, figsize=(12, 10.5))
ax = ax.flatten()
for layer in layers:
    val_scores = NN_arch_data.loc[NN_arch_data['param_module__hidden_layers'] == layer, ['mean_test_score']].values
    ax[0].plot(units, val_scores, label='layers = '+str(layer))
ax[0].set_xlabel('Number of units per layer')
ax[0].set_ylabel('Validation accuracy')
ax[0].set_xscale('log', base=2)
ax[0].set_ylim([0.7, 0.95])
ax[0].legend()
plt.rcParams.update({'font.size': 14})
ax[0].xaxis.set_major_formatter(ScalarFormatter())

for dp in dps:
    val_scores = NN_reg_data.loc[NN_reg_data['param_module__dropout_percent'] == dp, ['mean_test_score']].values
    ax[1].plot(decay, val_scores, label='dp = '+str(dp))
ax[1].set_xlabel('Weight decay')
ax[1].set_ylabel('Validation accuracy')
ax[1].set_xscale('log', base=10)
ax[1].set_ylim([0.7, 0.95])
ax[1].legend()
plt.rcParams.update({'font.size': 14})

NN_lr_data = pd.read_excel('NN_bean_lr2.xlsx')

lrs = [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]
for i in range(7):
    ax[2].plot(NN_lr_data.iloc[:100, 2*i+1], label='lr = '+str(lrs[i]))
ax[2].set_xlabel('Iteration')
ax[2].set_ylabel('Training accuracy')
ax[2].set_ylim([0.15, 0.95])
ax[2].legend()

for i in range(7):
    ax[3].plot(NN_lr_data.iloc[:100, 2*i+2], label='lr = '+str(lrs[i]))
ax[3].set_xlabel('Iteration')
ax[3].set_ylabel('Validation accuracy')
ax[3].set_ylim([0.15, 0.95])
ax[3].legend(loc=4)

plt.tight_layout()

# plt.savefig('NN_hyperparams.png', dpi=500)

In [ ]:
# Final NN results

NN_best = NeuralNetClassifier(
    module=BackpropModule,
    module__input_dim=num_features,
    module__output_dim=num_classes,
    module__hidden_units=64,
    module__hidden_layers=1,
    module__dropout_percent=0.1,
    max_epochs=100,
    verbose=0,
    callbacks=[EpochScoring(scoring='accuracy', name='train_acc', on_train=True),],
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.SGD,
    optimizer__weight_decay=0,
    optimizer__momentum=0.9,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

NN_best.fit(X_train, y_train)

# plot the iterative learning curve (accuracy)
ax[0].plot(NN_best.history[:, 'train_acc'], label='Train Acc', color='cornflowerblue')
ax[0].plot(NN_best.history[:, 'valid_acc'], label='Validation Acc', color='chartreuse')
ax[0].set_xlabel("Iteration")
ax[0].set_ylabel("Accuracy")
ax[0].legend()

train_sizes, train_scores, val_scores = learning_curve(NN_best, X_train, y_train, train_sizes=[0.025, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5, 0.75, 1], cv=folds)

train_scores_mean = train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
val_scores_mean = val_scores.mean(axis=1)
val_scores_std = val_scores.std(axis=1)

ax[1].fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color='green')
ax[1].fill_between(train_sizes, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, alpha=0.1, color='darkorchid')
ax[1].plot(train_sizes, train_scores_mean, label="Training Acc", color='cyan')
ax[1].plot(train_sizes, val_scores_mean, label="Validation Acc", color='darkorchid')
ax[1].set_xlabel("Training size")
ax[1].set_ylabel("Accuracy")
ax[1].legend()

plt.tight_layout()

# plt.savefig('NN_final_trains.png', dpi=500)

In [ ]:
# test accuracy
y_test_prob = NN_best.predict_proba(X_test)
y_test_pred = np.argmax(y_test_prob, axis=1)
NN_test_acc = metrics.accuracy_score(y_test, y_test_pred)
NN_test_recall = metrics.recall_score(y_test, y_test_pred, average='macro')
NN_test_f1 = metrics.f1_score(y_test, y_test_pred, average='macro')
print('Neural network on wine dataset, test accuracy is: {:.3f}'.format(NN_test_acc))
print('Neural network on wine dataset, test recall is: {:.3f}'.format(NN_test_recall))
print('Neural network on wine dataset, test f1 score is: {:.3f}'.format(NN_test_f1))

# confusion matrix
cm = metrics.confusion_matrix(y_test_pred, y_test)
cm_plot = metrics.ConfusionMatrixDisplay(cm)
cm_plot.plot()
# cm_plot.figure_.savefig('NN_conf_mat.png', dpi=500)

%timeit NN_best.fit(X_train, y_train)
%timeit NN_best.predict(X_test)

In [ ]:
# Perform grid search with SVM

SVM = svm.SVC(max_iter=1000000000)

params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf'],
    'degree': [2, 3, 4],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001]
}

folds = StratifiedKFold(n_splits=5)
gs_SVM = GridSearchCV(SVM, params, refit=False, cv=folds, scoring='accuracy', verbose=3, n_jobs=12)

gs_SVM.fit(X_train, y_train)

df = pd.DataFrame(gs_SVM.cv_results_)
df.to_excel('SVM_bean_GS.xlsx')

In [ ]:
# Extract best params for SVM 

SVM_best_param = gs_SVM.best_params_
print(SVM_best_param)

In [ ]:
# hyperparam effects

SVM_GS_data = pd.read_excel('SVM_bean_GS.xlsx')
SVM_GS_kernel = SVM_GS_data.loc[(SVM_GS_data['param_C'] == 10) & (SVM_GS_data['param_gamma'] == 0.1), ['param_degree', 'param_kernel', 'mean_test_score']]
kernels = ['linear', 'poly_2', 'poly_3', 'poly_4', 'rbf']
scores = []
scores.append(SVM_GS_kernel.loc[SVM_GS_kernel['param_kernel'] == 'linear', ['mean_test_score']].values[0].item())
for i in [2, 3, 4]:
    scores.append(SVM_GS_kernel.loc[(SVM_GS_kernel['param_kernel'] == 'poly') & (SVM_GS_kernel['param_degree'] == i), ['mean_test_score']].values[0].item())
scores.append(SVM_GS_kernel.loc[SVM_GS_kernel['param_kernel'] == 'rbf', ['mean_test_score']].values[0].item())

SVM_time = svm.SVC(C=10, gamma=0.1, kernel='linear')
%timeit SVM_time.fit(X_train, y_train)
SVM_time = svm.SVC(C=10, gamma=0.1, kernel='poly', degree=2)
%timeit SVM_time.fit(X_train, y_train)
SVM_time = svm.SVC(C=10, gamma=0.1, kernel='poly', degree=3)
%timeit SVM_time.fit(X_train, y_train)
SVM_time = svm.SVC(C=10, gamma=0.1, kernel='poly', degree=4)
%timeit SVM_time.fit(X_train, y_train)
SVM_time = svm.SVC(C=10, gamma=0.1, kernel='rbf')
%timeit SVM_time.fit(X_train, y_train)
fit_times = [418, 369, 289, 458, 346]

SVM_poly3 = svm.SVC(kernel='poly', degree=3, max_iter=1000000000)
params = {'C': [1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]}
SVM_C = GridSearchCV(SVM_poly3, params, refit=False, cv=folds, scoring='accuracy', verbose=3, n_jobs=12)
SVM_C.fit(X_train, y_train)
df = pd.DataFrame(SVM_C.cv_results_)
df.to_excel('SVM_bean_C.xlsx')

In [ ]:
SVM_best = svm.SVC(kernel='poly', degree=3, C=10)

train_sizes, train_scores, val_scores = learning_curve(SVM_best, X_train, y_train, train_sizes=np.linspace(0.05, 1, 20), cv=folds)

train_scores_mean = train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
val_scores_mean = val_scores.mean(axis=1)
val_scores_std = val_scores.std(axis=1)

In [ ]:
SVM_C_data = pd.read_excel('SVM_bean_C.xlsx')

kernels = ['linear', 'poly_2', 'poly_3', 'poly_4', 'rbf']

Cs = SVM_C_data['param_C']
C_scores = SVM_C_data['mean_test_score']
C_times = SVM_C_data['mean_fit_time'] * 1000

fig, ax = plt.subplots(2, 2, figsize=(12, 10.5))
ax = ax.flatten()
ax[0].bar(kernels, scores, label=kernels)
ax[0].set_xlabel('Kernel type')
ax[0].set_ylabel('Validation accuracy')
ax[0].set_ylim([0, 1])
plt.rcParams.update({'font.size': 14})

ax[1].bar(kernels, fit_times, label=kernels, color='orange')
ax[1].set_xlabel('Kernel type')
ax[1].set_ylabel('Model fit runtime (ms)')

ln1 = ax[2].plot(Cs, C_scores, label='Accuracy')
ax[2].set_xlabel('C')
ax[2].set_ylabel('Validation accuracy')
ax[2].set_xscale('log', base=10)
ax[2].set_ylim([0.25, 1.05])
ax2 = ax[2].twinx()
ln2 = ax2.plot(Cs, C_times, color='orange', label='Runtime')
ax2.set_ylabel('Model fit runtime (ms)')
lns = ln1 + ln2
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=0)

ax[3].fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color='green')
ax[3].fill_between(train_sizes, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, alpha=0.1, color='darkorchid')
ax[3].plot(train_sizes, train_scores_mean, label="Training score", color='cyan')
ax[3].plot(train_sizes, val_scores_mean, label="Validation score", color='darkorchid')
ax[3].set_xlabel("Training size")
ax[3].set_ylabel("Score")
ax[3].legend()

plt.tight_layout()

# plt.savefig('SVM_results.png', dpi=500)

In [ ]:
# test accuracy
SVM_best.fit(X_train, y_train)
y_test_pred = SVM_best.predict(X_test)

SVM_test_acc = metrics.accuracy_score(y_test, y_test_pred)
SVM_test_recall = metrics.recall_score(y_test, y_test_pred, average='macro')
SVM_test_f1 = metrics.f1_score(y_test, y_test_pred, average='macro')
print('SVM on wine dataset, test accuracy is: {:.3f}'.format(SVM_test_acc))
print('SVM on wine dataset, test recall is: {:.3f}'.format(SVM_test_recall))
print('SVM on wine dataset, test f1 score is: {:.3f}'.format(SVM_test_f1))

# confusion matrix
cm = metrics.confusion_matrix(y_test_pred, y_test)
cm_plot = metrics.ConfusionMatrixDisplay(cm)
cm_plot.plot()
# cm_plot.figure_.savefig('SVM_conf_mat.png', dpi=500)

%timeit SVM_best.fit(X_train, y_train)
%timeit SVM_best.predict(X_test)

In [ ]:
# Perform grid search with kNN

kNN = neighbors.KNeighborsClassifier()

params = {
    'n_neighbors': np.linspace(1, 30, 30, dtype='int').tolist(),
    'metric': ['euclidean', 'manhattan', 'cosine'],
    'weights': ['uniform', 'distance']
}

folds = StratifiedKFold(n_splits=5)
gs_kNN = GridSearchCV(kNN, params, refit=False, cv=folds, scoring='accuracy', verbose=3, n_jobs=12)

gs_kNN.fit(X_train, y_train)

df = pd.DataFrame(gs_kNN.cv_results_)
df.to_excel('kNN_bean_GS.xlsx')

In [ ]:
# hyperparam effects

kNN_GS_data = pd.read_excel('kNN_bean_GS.xlsx')
k = np.linspace(1, 30, 30)
euc = kNN_GS_data.loc[(kNN_GS_data['param_weights'] == 'uniform') & (kNN_GS_data['param_metric'] == 'euclidean'), ['mean_test_score']].values
man = kNN_GS_data.loc[(kNN_GS_data['param_weights'] == 'uniform') & (kNN_GS_data['param_metric'] == 'manhattan'), ['mean_test_score']].values
cos = kNN_GS_data.loc[(kNN_GS_data['param_weights'] == 'uniform') & (kNN_GS_data['param_metric'] == 'cosine'), ['mean_test_score']].values

fig, ax = plt.subplots(2, 2, figsize=(12, 10.5))
ax = ax.flatten()

ax[0].plot(k, euc, label="euclidean")
ax[0].plot(k, man, label="manhattan")
ax[0].plot(k, cos, label="cosine", color='red')
ax[0].set_xlabel("k")
ax[0].set_ylabel("Validation accuracy")
ax[0].set_ylim([0.875, 0.928])
ax[0].legend()

euc = kNN_GS_data.loc[(kNN_GS_data['param_weights'] == 'distance') & (kNN_GS_data['param_metric'] == 'euclidean'), ['mean_test_score']].values
man = kNN_GS_data.loc[(kNN_GS_data['param_weights'] == 'distance') & (kNN_GS_data['param_metric'] == 'manhattan'), ['mean_test_score']].values
cos = kNN_GS_data.loc[(kNN_GS_data['param_weights'] == 'distance') & (kNN_GS_data['param_metric'] == 'cosine'), ['mean_test_score']].values

ax[1].plot(k, euc, label="euclidean")
ax[1].plot(k, man, label="manhattan")
ax[1].plot(k, cos, label="cosine", color='red')
ax[1].set_xlabel("k")
ax[1].set_ylabel("Validation accuracy")
ax[1].set_ylim([0.875, 0.928])
ax[1].legend()

kNN_best = neighbors.KNeighborsClassifier(n_neighbors=22, metric='euclidean')
kNN_best.fit(X_train, y_train)

train_sizes, train_scores, val_scores = learning_curve(kNN_best, X_train, y_train, train_sizes=np.linspace(0.05, 1.0, 20), cv=5)

train_scores_mean = train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
val_scores_mean = val_scores.mean(axis=1)
val_scores_std = val_scores.std(axis=1)

ax[2].fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color='green')
ax[2].fill_between(train_sizes, val_scores_mean - val_scores_std, val_scores_mean + val_scores_std, alpha=0.1, color='darkorchid')
ax[2].plot(train_sizes, train_scores_mean, label="Training score", color='cyan')
ax[2].plot(train_sizes, val_scores_mean, label="Validation score", color='darkorchid')
ax[2].set_xlabel("Training size")
ax[2].set_ylabel("Score")
ax[2].set_ylim([0.882, 0.932])
ax[2].legend()

ax[3].axis('off')

plt.tight_layout()

# plt.savefig('kNN_results.png', dpi=500)

In [ ]:
# test accuracy
kNN_best.fit(X_train, y_train)
y_test_pred = kNN_best.predict(X_test)
kNN_test_acc = metrics.accuracy_score(y_test, y_test_pred)
kNN_test_recall = metrics.recall_score(y_test, y_test_pred, average='macro')
kNN_test_f1 = metrics.f1_score(y_test, y_test_pred, average='macro')
print('kNN on wine dataset, test accuracy is: {:.3f}'.format(kNN_test_acc))
print('kNN on wine dataset, test recall is: {:.3f}'.format(kNN_test_recall))
print('kNN on wine dataset, test f1 score is: {:.3f}'.format(kNN_test_f1))
# confusion matrix
cm = metrics.confusion_matrix(y_test_pred, y_test)
cm_plot = metrics.ConfusionMatrixDisplay(cm)
cm_plot.plot()
plt.rcParams.update({'font.size': 14})
cm_plot.figure_.savefig('kNN_conf_mat.png', dpi=500)

%timeit kNN_best.fit(X_train, y_train)
%timeit kNN_best.predict(X_test)